In [1]:
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

filename = next(iter(uploaded))

if filename.lower().endswith((".xlsx", ".xls")):
    data = pd.read_excel(io.BytesIO(uploaded[filename]), sheet_name=0, engine="openpyxl")
else:
    data = pd.read_csv(io.BytesIO(uploaded[filename]))

print("Loaded:", filename, data.shape)
data.head()


Saving HSI_HSIF0923_calendar_filled.xlsx to HSI_HSIF0923_calendar_filled (3).xlsx
Loaded: HSI_HSIF0923_calendar_filled (3).xlsx (38, 4)


,Date,HSI,HSIF0923,IsTradingDay
0,2023-08-01,20011,20035,True
1,2023-08-02,19517,19537,True
2,2023-08-03,19421,19440,True
3,2023-08-04,19539,19523,True
4,2023-08-05,19539,19523,False


In [3]:
import math
import pandas as pd

N_CONTRACTS = 8
IM, MM = 101_944.0, 81_555.0
R = 0.02                                # annual cont. compounding
MULTIPLIER = 50.0                       # HSI futures multiplier
FEE_PER_SIDE = 110.54
COL_FUT = "HSIF0923"                     # futures price column

data = data.sort_values("Date").reset_index(drop=True)
data["Date"] = pd.to_datetime(data["Date"])
data["dF"] = data[COL_FUT].diff().fillna(0.0)

# commision deal
data["Fee"] = 0.0
data.loc[data.index[0], "Fee"] = N_CONTRACTS * FEE_PER_SIDE
data.loc[data.index[-1], "Fee"] = N_CONTRACTS * FEE_PER_SIDE

# interest+margin by mtm
init_total  = N_CONTRACTS * IM
maint_total = N_CONTRACTS * MM
daily_factor = math.exp(R / 365.0)

balances, deposits, interests, vms = [], [], [], []

# Day 1: after opening fee
B_prev = init_total - data.loc[0, "Fee"]
balances.append(B_prev); deposits.append(0.0); interests.append(0.0); vms.append(0.0)

#interating
for i in range(1, len(data)):
    interest = B_prev * (daily_factor - 1.0)                  # daily interest (weekends included)
    vm = N_CONTRACTS * MULTIPLIER * data.loc[i, "dF"]         # MTM vs previous calendar day
    B_tmp = B_prev + interest + vm - data.loc[i, "Fee"]

    deposit = 0.0
    if B_tmp < maint_total:                                   # margin call -> top up to init_total
        deposit = init_total - B_tmp
        B_tmp += deposit

    interests.append(interest); vms.append(vm); deposits.append(deposit); balances.append(B_tmp)
    B_prev = B_tmp

# generate table
q1_table = pd.DataFrame({
    "Date": data["Date"].dt.date,
    "FuturesPrice": data[COL_FUT],
    "dF": data["dF"],
    "VM": vms,
    "Interest": interests,
    "Fee": data["Fee"],
    "Deposit": deposits,
    "Balance": balances
})
q1_table[["dF","VM","Interest","Fee","Deposit","Balance"]] = q1_table[["dF","VM","Interest","Fee","Deposit","Balance"]].round(2)

q1_table.to_csv("Q1_margin_table.csv", index=False, encoding="utf-8-sig")
print("Saved: Q1_margin_table.csv   Shape:", q1_table.shape)
q1_table



Saved: Q1_margin_table.csv   Shape: (38, 8)


,Date,FuturesPrice,dF,VM,Interest,Fee,Deposit,Balance
0,2023-08-01,20035,0.0,0.0,0.00,884.32,0.00,814667.68
1,2023-08-02,19537,-498.0,-199200.0,44.64,0.00,200039.68,815552.00
2,2023-08-03,19440,-97.0,-38800.0,44.69,0.00,0.00,776796.69
3,2023-08-04,19523,83.0,33200.0,42.57,0.00,0.00,810039.25
4,2023-08-05,19523,0.0,0.0,44.39,0.00,0.00,810083.64
5,2023-08-06,19523,0.0,0.0,44.39,0.00,0.00,810128.03
6,2023-08-07,19525,2.0,800.0,44.39,0.00,0.00,810972.42
7,2023-08-08,19136,-389.0,-155600.0,44.44,0.00,0.00,655416.86
8,2023-08-09,19236,100.0,40000.0,35.91,0.00,0.00,695452.77
9,2023-08-10,19248,12.0,4800.0,38.11,0.00,0.00,700290.88


In [4]:
# Q2: Annualized rate of return

# final balance from Q1 table
final_balance = q1_table["Balance"].iloc[-1]

# settlement fee: 10 HKD per contract * 8 contracts = 80
settlement_fee = 10.0 * N_CONTRACTS
final_balance_net = final_balance - settlement_fee

# total invested = initial margin + all deposits
initial_cash = N_CONTRACTS * IM
total_deposits = q1_table["Deposit"].sum()
total_invested = initial_cash + total_deposits

# net profit and holding period return
net_profit = final_balance_net - total_invested
hpr = net_profit / total_invested

# investment horizon in days
days = (q1_table["Date"].iloc[-1] - q1_table["Date"].iloc[0]).days

# annualized return
annualized_return = (1 + hpr) ** (365 / days) - 1

print("Total Invested:", round(total_invested, 2))
print("Final Balance (after settlement fee):", round(final_balance_net, 2))
print("Net Profit:", round(net_profit, 2))
print("Holding Period Return:", round(hpr*100, 2), "%")
print("Annualized Return:", round(annualized_return*100, 2), "%")


Total Invested: 1684920.89
Final Balance (after settlement fee): 933193.86
Net Profit: -751727.03
Holding Period Return: -44.61 %
Annualized Return: -99.71 %


In [6]:
# Q3: Compare futures return vs direct HSI investment

# spot HSI start and end
hsi_start = data["HSI"].iloc[0]
hsi_end   = data["HSI"].iloc[-1]

# holding period return (spot)
hsi_hpr = (hsi_end - hsi_start) / hsi_start
hsi_annualized = (1 + hsi_hpr) ** (365 / days) - 1

print("HSI Start:", hsi_start)
print("HSI End:", hsi_end)
print("HSI Holding Period Return:", round(hsi_hpr*100, 2), "%")
print("HSI Annualized Return:", round(hsi_annualized*100, 2), "%")




HSI Start: 20011
HSI End: 18202
HSI Holding Period Return: -9.04 %
HSI Annualized Return: -60.73 %
